<a href="https://colab.research.google.com/github/sidagarwal-labs/DSBA-6211---Advance-Business-Analytics/blob/main/PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#get the notebook, env ready
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#read the data
df=pd.read_csv('Fundraising.csv')
df.describe(include='all')

,zipcode,homeowner dummy,NUMCHLD,gender dummy,WEALTH,HV,Icmed,Icavg,NUMPROM,MAXRAMNT,totalmonths,Donor
count,3120.000000,3120.000000,3120.000000,3120.000000,3120.000000,3120.000000,3120.000000,3120.000000,3120.000000,3120.000000,3120.000000,3120.000000
mean,3.766667,0.770192,1.069231,0.609295,6.402244,1141.361859,388.217308,432.088141,49.089423,16.651397,31.136859,0.383654
std,1.176904,0.420777,0.347688,0.487987,2.539978,946.642162,172.815950,168.195104,22.717130,22.223521,4.132952,0.486353
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,5.000000,17.000000,0.000000
25%,3.000000,1.000000,1.000000,0.000000,5.000000,556.000000,278.000000,318.000000,29.000000,10.000000,29.000000,0.000000
50%,4.000000,1.000000,1.000000,1.000000,8.000000,822.000000,356.000000,396.000000,48.000000,15.000000,31.000000,0.000000
75%,5.000000,1.000000,1.000000,1.000000,8.000000,1338.750000,465.000000,516.000000,65.000000,20.000000,34.000000,1.000000
max,5.000000,1.000000,5.000000,1.000000,9.000000,5945.000000,1500.000000,1331.000000,157.000000,1000.000000,37.000000,1.000000


In [ ]:
#check the data type and missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3120 entries, 0 to 3119
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   zipcode          3120 non-null   int64  
 1   homeowner dummy  3120 non-null   int64  
 2   NUMCHLD          3120 non-null   int64  
 3   gender dummy     3120 non-null   int64  
 4   WEALTH           3120 non-null   int64  
 5   HV               3120 non-null   int64  
 6   Icmed            3120 non-null   int64  
 7   Icavg            3120 non-null   int64  
 8   NUMPROM          3120 non-null   int64  
 9   MAXRAMNT         3120 non-null   float64
 10  totalmonths      3120 non-null   int64  
 11  Donor            3120 non-null   int64  
dtypes: float64(1), int64(11)
memory usage: 292.6 KB


In [ ]:
X=df.iloc[:,0:11]
y=df['Donor']

In [ ]:
#rescale all the independent variables for our models. ((X - mean) / st dev)
from sklearn.preprocessing import StandardScaler
X=StandardScaler().fit_transform(X)

In [ ]:
#data partition
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
#add logistic regression model
#model 1 is simple log reg on training dataset
from sklearn.linear_model import LogisticRegression
model1=LogisticRegression()
model1.fit(X_train,y_train)

LogisticRegression()

In [ ]:
#use accuracy as a benchmark to compare perf across models
model1.score(X_val,y_val)

0.6677350427350427

In [ ]:
#creating the 2nd model
#adding all X independent variables into the first PCA
#11 variables = 11 features = 11 principal components
from sklearn.decomposition import PCA
pca=PCA()
PCA_V1=pca.fit_transform(X)

In [ ]:
#these are the principal components for first observation
print(PCA_V1[0,:])

[ 1.85357544  0.77740602  1.11342648  0.57636915 -0.70724986  0.447121
 -1.1719957   0.48479383  0.88129526 -0.41904927  0.13945821]


In [ ]:
#getting the eigenvalues, eigenvector, variance % for the principal components
#below is % of variation captured
#1st = 0.2699 or 27%, 2nd = 0.1329 or 13.3%, etc....
#this is pretty bad. Likely because we are using all the variables in 1 set.
#this is leading to a lot of different variables explaining different concepts.
pca.explained_variance_ratio_

array([0.26993694, 0.13296439, 0.09755934, 0.09314764, 0.08973188,
       0.08819136, 0.08415172, 0.07559969, 0.04144892, 0.02479158,
       0.00247653])

In [ ]:
#these are eigenvalues
#we keep values / principal components with a value greater than 1.
#so we keep the first 4, and this means from the ratios above (27+13+9.7+9.3)
#it is roughly 57-58% of variance explained. Rest is dropped.
pca.explained_variance_

array([2.97025835, 1.46307725, 1.07349682, 1.0249525 , 0.98736713,
       0.97041598, 0.92596574, 0.83186322, 0.45608434, 0.27279486,
       0.02725058])

In [ ]:
#how the principal components are constructed
pca.components_

array([[ 1.88599819e-01,  1.17824379e-01,  3.95630864e-02,
        -2.55922182e-02,  3.05520130e-01,  4.95865549e-01,
         5.42436063e-01,  5.45846038e-01, -1.17231713e-01,
         2.85404254e-02,  4.85749173e-02],
       [ 2.12015991e-01,  8.39969403e-03, -1.93860556e-01,
         1.18355250e-01, -4.15048984e-01,  1.64232593e-01,
         1.01382691e-01,  9.81517530e-02,  6.78913772e-01,
         2.22738897e-01, -4.16775765e-01],
       [-3.93420510e-01,  3.03826675e-01,  5.71293274e-01,
        -1.26578865e-01,  1.13129707e-01, -1.06434470e-01,
         7.74372807e-02,  7.55164465e-02,  1.64011438e-01,
        -3.00793477e-01, -5.07256423e-01],
       [ 2.92403785e-01,  2.69636918e-01, -1.85129018e-01,
         6.29737914e-01,  2.62507626e-02,  2.49895944e-02,
        -6.73064953e-02, -6.54040727e-02, -9.88379462e-02,
        -6.14089715e-01, -1.18138934e-01],
       [ 2.62236187e-01,  7.30347904e-01,  1.48007301e-01,
        -3.08066802e-01, -3.16362249e-01,  2.80026751e-02,
  

In [ ]:
#rerun model and only keep top 4 components
#fit the PCA V1
pca=PCA(n_components=4)
PCA_V1=pca.fit_transform(X)

In [ ]:
#create new data partition for model 2
#using the new PCA_V1 which is top 4 features
X2_train, X2_val, y2_train, y2_val = train_test_split(PCA_V1, y, test_size=0.3, random_state=0)

In [ ]:
#build model 2 which is again log reg
#see how accuracy / score stays mainly high even with only leveraging top 4 variables instead of all 11.
model2=LogisticRegression()
model2.fit(X2_train,y2_train)
model2.score(X2_val,y2_val)

0.6613247863247863

In [ ]:
#Check correlation of independent variables
#ICMed, ICAverage, HV are all correlated. Which makes sense.
#These all explain financial condition of a donor
df.corr()

,zipcode,homeowner dummy,NUMCHLD,gender dummy,WEALTH,HV,Icmed,Icavg,NUMPROM,MAXRAMNT,totalmonths,Donor
zipcode,1.000000,0.040594,-0.036512,0.024321,0.016285,0.358698,0.168491,0.170888,-0.005263,0.021753,-0.001211,0.027054
homeowner dummy,0.040594,1.000000,0.045229,0.002911,0.064920,0.111643,0.138074,0.133026,0.003761,-0.029656,0.025650,0.058092
NUMCHLD,-0.036512,0.045229,1.000000,-0.033273,0.061760,-0.010206,0.050766,0.050905,-0.087367,-0.017954,-0.005034,-0.037672
gender dummy,0.024321,0.002911,-0.033273,1.000000,-0.030954,-0.022055,-0.025058,-0.026248,0.033954,0.002595,-0.036908,0.202195
WEALTH,0.016285,0.064920,0.061760,-0.030954,1.000000,0.258350,0.373116,0.381161,-0.413888,-0.024830,0.027617,-0.037240
HV,0.358698,0.111643,-0.010206,-0.022055,0.258350,1.000000,0.733773,0.748237,-0.062467,0.055236,0.026103,0.005373
Icmed,0.168491,0.138074,0.050766,-0.025058,0.373116,0.733773,1.000000,0.972409,-0.050267,0.046222,0.034636,-0.023449
Icavg,0.170888,0.133026,0.050905,-0.026248,0.381161,0.748237,0.972409,1.000000,-0.056079,0.042985,0.032965,-0.030679
NUMPROM,-0.005263,0.003761,-0.087367,0.033954,-0.413888,-0.062467,-0.050267,-0.056079,1.000000,0.110307,-0.284819,0.081658
MAXRAMNT,0.021753,-0.029656,-0.017954,0.002595,-0.024830,0.055236,0.046222,0.042985,0.110307,1.000000,0.019870,0.020387


In [ ]:
#turn X3 into a dataframe X, specify columan names
X3=pd.DataFrame(data=X,
                columns=['zip',
                         'homeowner',
                         'child',
                         'gender',
                         'wealth',
                         'HV',
                         'icmed',
                         'icavg',
                         'num',
                         'max',
                         'totalm'])

In [ ]:
#split the data into 2 parts
#first data = all the variables not correlated
#second data (candidate variable) = all the correlated variables HV, ICMED, ICAvg
X3_no_PCA=pd.concat([X3.iloc[:,0:5],X3.iloc[:,8:11]],axis=1)
X3_no_PCA.head()

,zip,homeowner,child,gender,wealth,num,max,totalm
0,-0.651531,0.546239,-0.199149,0.800776,1.022912,1.096731,-0.479362,-0.517113
1,-1.501355,0.546239,-0.199149,-1.248789,0.235377,-0.136017,-0.209334,0.692870
2,1.048116,-1.830701,2.677454,0.800776,0.629144,-0.752391,-0.299343,-0.517113
3,1.048116,0.546239,-0.199149,-1.248789,-0.945925,1.977265,-0.299343,-0.275116
4,-0.651531,0.546239,-0.199149,-1.248789,0.629144,-1.280711,-0.254339,-0.275116


In [ ]:
#generate second part which is 3 correlated columns
X3_PCA=X3.iloc[:,5:8]
X3_PCA.head()

,HV,icmed,icavg
0,0.272204,1.439813,1.610958
1,-0.468427,0.195515,0.183815
2,-0.331078,-0.174881,-0.333524
3,0.348274,0.554336,0.677369
4,-0.627964,-0.012833,-0.000524


In [ ]:
#now lets run PCA analysis
pca2=PCA()
PCA_V2=pca2.fit_transform(X3_PCA)

In [ ]:
#check the % of variance explained
#wow look at this. First level captures 97%!
pca2.explained_variance_ratio_

array([0.88048423, 0.11040409, 0.00911168])

In [ ]:
#check the eigenvalue, we only keep greater than 1
pca2.explained_variance_

array([2.64229957, 0.33131846, 0.02734382])

In [ ]:
#check PCA2 components
pca2.components_

array([[ 0.53811795,  0.59450807,  0.59748575],
       [ 0.84249714, -0.40045738, -0.36032271],
       [-0.02505282, -0.69727615,  0.71636466]])

In [ ]:
#so we keep 1 component
pca2=PCA(n_components=1)
PCA_V2=pca2.fit_transform(X3_PCA)

In [ ]:
pcaDF=pd.DataFrame(data=PCA_V2,columns=['PC1'])

In [ ]:
#So we replaced all the 3 correlated values with a 1 component PCA value that explains a large of the variance
X3_model=pd.concat([X3_no_PCA,pcaDF],axis=1)
X3_model.head()

,zip,homeowner,child,gender,wealth,num,max,totalm,PC1
0,-0.651531,0.546239,-0.199149,0.800776,1.022912,1.096731,-0.479362,-0.517113,1.964982
1,-1.501355,0.546239,-0.199149,-1.248789,0.235377,-0.136017,-0.209334,0.692870,-0.026007
2,1.048116,-1.830701,2.677454,0.800776,0.629144,-0.752391,-0.299343,-0.517113,-0.481403
3,1.048116,0.546239,-0.199149,-1.248789,-0.945925,1.977265,-0.299343,-0.275116,0.921688
4,-0.651531,0.546239,-0.199149,-1.248789,0.629144,-1.280711,-0.254339,-0.275116,-0.345861


In [ ]:
#a new data partition
X3_train,X3_val,y3_train,y3_val=train_test_split(X3_model,y,test_size=0.3,random_state=0)

In [ ]:
#model 3 score using the new PCA value
model3=LogisticRegression()
model3.fit(X3_train,y3_train)
model3.score(X3_val,y3_val)

0.6591880341880342

In [ ]:
pd.DataFrame(data=model3.coef_,columns=X3_model.columns)

,zip,homeowner,child,gender,wealth,num,max,totalm,PC1
0,0.037496,0.103533,-0.065559,0.425445,-0.026541,0.021834,0.052593,-0.352492,-0.02282
